In [1]:
import os
import random
import pandas as pd
import altair as alt
import shutil

In [2]:
df = pd.read_csv('ba-europe-countries.csv', index_col=0)
df.head()

,Albania,Austria,Belgium,Bulgaria,Bosnia and Herzegovina,Belarus,Switzerland,Czech Republic,Germany,Denmark,...,Norway,Poland,Portugal,Romania,Republic of Serbia,Slovakia,Slovenia,Sweden,Ukraine,Europe
year,,,,,,,,,,,,,,,,,,,,,
2001,115110700.0,15992066.0,0.0,7.601922e+09,1.347520e+08,7.210384e+08,0.0,5795782.0,0.0,0.0,...,11054921.0,132981040.0,9.907034e+08,6.610198e+09,7.136327e+08,18245980.0,0.0,0.0,2.741712e+10,5.303158e+10
2002,21895182.0,9391316.0,0.0,9.800241e+08,3.262811e+07,2.961270e+09,0.0,26885992.0,2200249.0,0.0,...,4239507.0,309537760.0,1.260476e+09,7.933247e+08,3.195194e+08,50820430.0,0.0,87044070.0,1.952653e+10,3.086711e+10
2003,306317900.0,14274798.0,6708083.0,2.510970e+09,1.103024e+09,1.702243e+09,5098142.0,7942368.0,18192316.0,6117771.0,...,89941980.0,563908200.0,4.766764e+09,1.228760e+09,1.543557e+09,69442060.0,1502610.0,20446228.0,4.935378e+09,2.852175e+10
2004,46205270.0,21358532.0,321988.0,4.950672e+09,4.652726e+07,3.819851e+08,0.0,17548342.0,3595529.0,6064104.0,...,13201502.0,78994376.0,1.433652e+09,4.240313e+09,1.058536e+09,134805630.0,858634.0,5151806.0,1.353884e+10,3.414591e+10
2005,154876220.0,38155576.0,375652.0,2.299370e+09,5.768951e+07,3.666906e+08,0.0,36116316.0,0.0,2414909.0,...,37672590.0,89512640.0,3.957554e+09,3.162673e+09,6.935084e+08,88385700.0,4454167.0,37350600.0,2.363365e+10,3.979723e+10


In [3]:
df_perc = (100* df / df.sum()).round(0).reset_index()
df_perc = pd.melt(df_perc, "year")
country_list = df_perc['variable'].unique().tolist()

df_perc.tail()

,year,variable,value
775,2016,Europe,5.0
776,2017,Europe,6.0
777,2018,Europe,2.0
778,2019,Europe,4.0
779,2020,Europe,3.0


In [4]:
# create charts:
path = "graphics/bars"
isExist = os.path.exists(path)
if not isExist:
   os.makedirs(path)

path = "graphics/stripes"
isExist = os.path.exists(path)
if not isExist:
   os.makedirs(path)

for c in country_list:
  print(c)

  # bar chart
  df_viz = df_perc[df_perc['variable']==c]

  sum = df_viz['value'].sum()
  sheets = round(int(sum*16))

  chart_bar = alt.Chart(df_viz).mark_bar().encode(
      y=alt.Y('value', stack="normalize", axis=None),
      x=alt.X('variable', axis=None),
      color=alt.Color('value', legend=None, scale=alt.Scale(scheme='greys')),
      tooltip=['variable','year','value'],
      order=alt.Order(
        # Sort the segments of the bars by this field
        'year',
        sort='descending'
      )
      ).configure_axis(
      grid=False
      ).configure_view(
      strokeWidth=0
      ).properties(
      width=800,
      height=800
      ).properties(
        title = c.lower() + ' fire history - 2001/2020' 
          ).configure_title(
          fontSize=20,
          font='Helvetica',
          anchor='middle',
          color='black',
          fontWeight = 'lighter'
        ).configure_view(
          strokeWidth=0
        )
          
  filename_bar = 'graphics/bars/' + c + '.html' 
  chart_bar.save(filename_bar)

  # stripe chart
  values = df_viz['value'].to_list()

  # create data for strip plot
  try:
    i = 1
    viz_values = []
    viz_color_values = []
    for e in values:
      r = int(e*e*3)

      for f in range(0,r):
        try:
          h = i+e
          s = 100 - round(random.uniform(i,h), 3)
        except:
          pass
        viz_values.append(s)
        viz_color_values.append(e)

      i = i+e

    df_viz_2 = pd.DataFrame (viz_values, columns = ['values'])
    df_viz_2_color = pd.DataFrame (viz_color_values, columns = ['colors'])

    df_viz_2 = df_viz_2.join(df_viz_2_color)
    df_viz_2['city'] = 'city'
    df_viz_2 = df_viz_2[df_viz_2['values'].between(1, 100)] #filter for values >100 (rounding errors)

    chart_stripe = alt.Chart(df_viz_2).mark_tick(size=800, color='#333333', thickness=1).encode(
        y=alt.Y('values', axis=None),
        x=alt.X('city', axis=None),
        color=alt.Color('colors', legend=None, scale=alt.Scale(scheme='greys')),
    ).properties(
          width=800,
          height=800
          ).properties(
            title = c.lower() + ' fire history - 2001/2020' 
              ).configure_title(
              fontSize=20,
              font='Helvetica',
              anchor='middle',
              color='black',
              fontWeight = 'lighter'
            ).configure_view(
              strokeWidth=0
            )
              
    filename_html = 'graphics/stripes/' + c + '.html' 
    chart_stripe.save(filename_html)
  except:
    pass

Albania
Austria
Belgium
Bulgaria
Bosnia and Herzegovina
Belarus
Switzerland
Czech Republic
Germany
Denmark
Spain
Estonia
Finland
France
United Kingdom
Greece
Croatia
Hungary
Ireland
Iceland
Italy
Kosovo
Lithuania
Luxembourg
Latvia
Moldova
Macedonia
Montenegro
Netherlands
Norway
Poland
Portugal
Romania
Republic of Serbia
Slovakia
Slovenia
Sweden
Ukraine
Europe


In [5]:
shutil.make_archive('charts', 'zip', 'graphics')

'/content/charts.zip.zip'

In [6]:
chart_bar

alt.Chart(...)

In [7]:
chart_stripe

alt.Chart(...)